<a href="https://colab.research.google.com/github/akshataupadhye/SWB-GVCEH-Phase3/blob/main/sentiment_analysis/RedditSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
import pandas as pd

def sentiment_model_reddit(df):
    # Initialize sentiment analysis pipeline
    sentiment_analyzer = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1, truncation=True)
    sentiment_analyzer.tokenizer.model_max_length = 512

    # Initialize lists to store results
    sentiments = []
    scores = []

    # Process each post in the dataframe
    for post in df['TitleText']:
        # Segment the text
        segments = post.split('\n')

        # Initialize counters for sentiments and scores
        sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}
        sentiment_scores = {'positive': 0, 'negative': 0, 'neutral': 0}

        # Analyze sentiment for each segment
        for segment in segments:
            # Optionally, filter out segments that are too short or not meaningful
            if len(segment.strip()) > 0:
                result = sentiment_analyzer(segment)[0]
                sentiment_label = result['label']
                sentiment_score = result['score']

                # Update sentiment and score counters
                sentiment_counts[sentiment_label] += 1
                sentiment_scores[sentiment_label] += sentiment_score

        # Determine overall sentiment based on total scores
        if sum(sentiment_counts.values()) > 0:  # Check if any segment with non-zero sentiment scores
            overall_sentiment = max(sentiment_scores, key=sentiment_scores.get)
            overall_sentiment_score = sentiment_scores[overall_sentiment] / sentiment_counts[overall_sentiment]
        else:
            overall_sentiment = None
            overall_sentiment_score = None

        # Append results to lists
        sentiments.append(overall_sentiment)
        scores.append(overall_sentiment_score)

    # Add sentiment and score columns to the dataframe
    df['sentiment'] = sentiments
    df['score'] = scores

    print("Sentiment analysis complete!")
    return df


In [ ]:
test_df = pd.read_csv('/content/sample_set_sentiment.csv')

In [ ]:
test_df.head()

,Unnamed: 0,index,Subreddit,Title,Text,TitleText,manual_sentiment_label,Sentiment_Full,Sentence_Level_Sentiment_Compund,Relevent_Sentence_Sentiment_Compund,sentiment_TextBlob_Full,sentiment_TextBlob_Compound,relevant_sent_sentiment_TextBlob_Full,sentiment_bert_all,relevant_sent_bert_all
0,8197,8197,VictoriaBC,Relying on rental income in Oak Bay?,We are considering putting an offer on a house...,Relying on rental income in Oak Bay?. We are c...,Neutral,Neutral,Neutral,Neutral,Positive,Positive,Neutral,Negative,Negative
1,1259,1259,VictoriaBC,City Council opposes efforts to increase housi...,"[Link to article: In Fairfield, plans for smal...",City Council opposes efforts to increase housi...,Negative,Negative,Neutral,Neutral,Positive,Positive,Positive,Negative,Negative
2,10403,10403,britishcolumbia,B.C. housing minister tells people not to help...,NaN,B.C. housing minister tells people not to help...,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative
3,10606,10606,britishcolumbia,Vaughn Palmer: End of the line for single-fami...,NaN,Vaughn Palmer: End of the line for single-fami...,Neutral,Neutral,Neutral,Neutral,Positive,Positive,Positive,Positive,Positive
4,618,618,VictoriaBC,Will the housing market ever change for the be...,I’m a part of the younger generation and it se...,Will the housing market ever change for the be...,Negative,Positive,Positive,Neutral,Positive,Positive,Positive,Negative,Negative


In [ ]:
df = sentiment_model_reddit(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Sentiment analysis complete!


In [ ]:
df.to_csv("/content/sample_set_sentiment_scored.csv")